In [4]:
pip install pyscipopt

Note: you may need to restart the kernel to use updated packages.


In [5]:
from pyscipopt import Model, quicksum, multidict
import numpy as np
import pandas as pd
import geopandas as gpd
import random

In [6]:
# adapted from https://scipbook.readthedocs.io/en/latest/flp.html
def flp(I,J,d,M,c,existing_sites=None):
    model = Model("flp")
    x,y = {},{}
    for j in J:
        y[j] = model.addVar(vtype="B", name="y(%s)"%j)
        for i in I:
            x[i,j] = model.addVar(vtype="C", name="x(%s,%s)"%(i,j))
    for i in I:
        model.addCons(quicksum(x[i,j] for j in J) == d[i], "Demand(%s)"%i)
    for j in M:
        model.addCons(quicksum(x[i,j] for i in I) <= M[j]*y[j], "Capacity(%s)"%i)
    for (i,j) in x:
        model.addCons(x[i,j] <= d[i]*y[j], "Strong(%s,%s)"%(i,j))
    
    if existing_sites:
        for j in existing_sites:
            model.addCons(y[j] == 1, name=f"ForceOpen({j})")

    model.addCons(quicksum(y[j] for j in J) <= 6, "FacilityLimit") 
            
    model.setObjective(
        quicksum(c[i,j]*x[i,j] for i in I for j in J),
        "minimize")
    model.data = x,y
    return model

In [7]:
# for I, d make a dictionary of planning units to number of students
pu_data = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/pu_with_proj_SPLIT.geojson').set_index('pu_2324_84')
pu_data = pu_data['final_proj'].to_dict()

I, d = multidict(pu_data)

In [8]:
# for J, M make a dictionary of sites to capacities
schools = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/dps_hs_locations.geojson')
schools = schools.to_crs('EPSG:4326')
pu = gpd.read_file('/Users/leahwallihan/Durham_school_planning/geospatial files/pu_shape_new.geojson')
pu = pu.to_crs('EPSG:4326')

for i, geometry in enumerate(pu['geometry']):
    in_geometry = geometry.contains(schools['geometry'])
    pu_id = pu.loc[i, 'OBJECTID']

    schools.loc[in_geometry, 'pu'] = pu_id

In [22]:
pu.loc[397]

OBJECTID                                                      398
pu_2324_848                                                   398
X                                                   2026458.30083
Y                                                     816194.8785
M_min                                                         0.0
PS_ID                                                       193.0
PUID2122_2                                                    193
ps_id_833                                                   546.0
psid_982                                                    546.0
TIMS_PU                                                     PU193
Region                                                  Southwest
Shape_Length                                          3341.404385
Shape_Area                                           593801.56611
geometry        MULTIPOLYGON (((-78.91149635441231 35.99434749...
Name: 397, dtype: object

In [9]:
# let's remove planning units in the North from J to make problem simpler
not_north = pu[(pu['Region'] != 'Central')]

# initialize dictionary of planning units with capacity of 1600 for potential site
pu_dict = {}
for _, row in not_north.iterrows():
    pu_dict[row['OBJECTID']] = 1600

# find which planning units have existing school
schools['pu'] = None

for i, geometry in enumerate(pu['geometry']):
    in_geometry = schools.within(geometry)
    pu_id = pu.loc[i, 'OBJECTID']
    schools.loc[in_geometry, 'pu'] = pu_id

# replace capacities of planning units with existing schools
pu_dict[45] = 1600
pu_dict[507] = 1810
pu_dict[602] = 1540
pu_dict[566] = 1540
pu_dict[290] = 1535

J, M = multidict(pu_dict)

# define which sites already exist
existing_sites = {602, 290, 45, 566, 507}

In [10]:
# create distance matrix
c = {}

pu_centroids = pu.set_index('OBJECTID').geometry.centroid 

for i in I:
    for j in J:
        dist = pu_centroids[i].distance(pu_centroids[j])
        c[i, j] = dist

/var/folders/ml/wj07z83j0yq4rjs8n_7sv0mw0000gn/T/ipykernel_42466/1973786647.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  pu_centroids = pu.set_index('OBJECTID').geometry.centroid


In [11]:
 '''
# for testing:
I_small = random.sample(I, 100)
d_small = {i: d[i] for i in I_small}
c_small = {(i,j): c[i,j] for i in I_small for j in J if (i,j) in c}

model = flp(I_small, J, d_small, M, c_small, existing_sites=existing_sites)
model.setParam('limits/solutions', 3)
model.optimize()
EPS = 1.e-6
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]
print ("Optimal value=", model.getObjVal())
print ("Facilities at nodes:", facilities)
print ("Edges:", edges)
'''

'\n# for testing:\nI_small = random.sample(I, 100)\nd_small = {i: d[i] for i in I_small}\nc_small = {(i,j): c[i,j] for i in I_small for j in J if (i,j) in c}\n\nmodel = flp(I_small, J, d_small, M, c_small, existing_sites=existing_sites)\nmodel.setParam(\'limits/solutions\', 3)\nmodel.optimize()\nEPS = 1.e-6\nx,y = model.data\nedges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]\nfacilities = [j for j in y if model.getVal(y[j]) > EPS]\nprint ("Optimal value=", model.getObjVal())\nprint ("Facilities at nodes:", facilities)\nprint ("Edges:", edges)\n'

In [12]:
model = flp(I, J, d, M, c, existing_sites=existing_sites)
model.setParam('limits/solutions', 7)
model.optimize()
EPS = 1.e-6
x,y = model.data
edges = [(i,j) for (i,j) in x if model.getVal(x[i,j]) > EPS]
facilities = [j for j in y if model.getVal(y[j]) > EPS]
print ("Optimal value=", model.getObjVal())
print ("Facilities at nodes:", facilities)
print ("Edges:", edges)

presolving:
(round 1, fast)       142983 del vars, 143177 del conss, 0 add conss, 627192 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 2, fast)       142983 del vars, 146462 del conss, 0 add conss, 627192 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 1 clqs
(round 3, exhaustive) 142983 del vars, 146462 del conss, 0 add conss, 627192 chg bounds, 0 chg sides, 0 chg coeffs, 480925 upgd conss, 0 impls, 1 clqs
   (44527.2s) probing cycle finished: starting next cycle
   (44528.3s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (44529.0s) no symmetry present (symcode time: 0.23)
presolving (4 rounds: 4 fast, 2 medium, 2 exhaustive):
 142983 deleted vars, 146462 deleted constraints, 0 added constraints, 627192 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 352036368 implications, 1 cliques
presolved problem has 484941 variables (732 bin, 0 int, 0 impl, 484209 cont) and 48

In [13]:
solution_reports = []

# Get all stored solutions
sols = model.getSols()

for sidx, sol in enumerate(sols):
    assignments = {}

    for (i_, j_) in x:
        if model.getSolVal(sol, x[i_, j_]) > 0.5:
            if j_ not in assignments:
                assignments[j_] = []
            assignments[j_].append(i_)

    student_count = {}
    if 'students' in globals():  
        for j_, pus in assignments.items():
            student_count[j_] = sum(students.get(i_, 0) for i_ in pus)

    solution_reports.append({
        'solution_number': sidx + 1,
        'facilities': list(assignments.keys()),
        'assignments': assignments,
        'student_count': student_count if 'students' in globals() else None
    })

In [14]:
for report in solution_reports:
    print(f"\n--- Solution #{report['solution_number']} ---")
    print("Facilities opened:", report['facilities'])

    print("Assignments:")
    for fac, pus in report['assignments'].items():
        print(f"  Facility {fac} <-- Planning Units {pus}")

    if report['student_count']:
        print("Student Count per Facility:")
        for fac, count in report['student_count'].items():
            print(f"  Facility {fac}: {count} students")


--- Solution #1 ---
Facilities opened: [45, 290, 507, 516, 566, 602]
Assignments:
  Facility 45 <-- Planning Units [2, 19, 28, 29, 30, 52, 55, 58, 64, 65, 71, 85, 141, 144, 156, 157, 164, 165, 177, 178, 191, 192, 193, 196, 197, 199, 205, 206, 209, 210, 211, 213, 214, 215, 237, 238, 239, 242, 243, 323, 326, 331, 339, 340, 341, 344, 347, 348, 352, 353, 354, 368, 369, 370, 371, 377, 403, 404, 411, 416, 420, 429, 435, 437, 438, 439, 440, 442, 443, 451, 453, 463, 464, 483, 488, 489, 495, 508, 511, 531, 542, 548, 549, 550, 572, 574, 576, 581, 582, 583, 584, 585, 588, 589, 590, 591, 592, 594, 595, 596, 720, 721, 727, 736, 746, 766, 768, 770, 775, 783, 797, 811, 812, 813, 815, 816, 817, 828, 848]
  Facility 290 <-- Planning Units [8, 10, 12, 32, 34, 37, 46, 47, 79, 80, 97, 98, 99, 100, 114, 115, 116, 170, 254, 267, 269, 275, 278, 279, 284, 285, 286, 288, 289, 290, 291, 292, 293, 295, 296, 297, 298, 307, 308, 309, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 328, 373, 374, 375, 380, 382, 

In [18]:
'''
pu_to_facility = {}
for facility, pu_list in assignments.items():
    for pu_id in pu_list:
        pu_to_facility[pu_id] = facility

pu['assignment'] = pu['OBJECTID'].map(pu_to_facility)
'''

"\npu_to_facility = {}\nfor facility, pu_list in assignments.items():\n    for pu_id in pu_list:\n        pu_to_facility[pu_id] = facility\n\npu['assignment'] = pu['OBJECTID'].map(pu_to_facility)\n"

In [ ]:
# pu.to_file('CFLP_solution.geojson', driver='GeoJSON')

In [ ]:
schools

In [ ]:
# new_pu = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/pu_2324_SPLIT.geojson', driver='GeoJSON')